# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate work to ensure that the market data for AAPL is continuously monitored and analyzed effectively. I also need to ask the right questions to gather insights and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Utilize statistical modeling and machine learning to identify trends and predict market movements"}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL in order to monitor and analyze trends accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) S

Final Answer: The current stock price for Apple Inc. (AAPL) is $214.29, with a decrease of $2.38 (-1.10%). The market cap is $3.286T. Key events include earnings date from August 1, 2024, to August 5, 2024, and a forward dividend yield of 1.00 (0.47%). The company overview includes information on their products, services, and financial highlights. Additionally, the performance overview compares AAPL to the S&P 500 in terms of total returns.

> Finished chain.
 

The current stock price for Apple Inc. (AAPL) is $214.29, with a decrease of $2.38 (-1.10%). The market cap is $3.286T. Key events include earnings date from August 1, 2024, to August 5, 2024, and a forward dividend yield of 1.00 (0.47%). The company overview includes information on their products, services, and financial highlights. Additionally, the performance overview compares AAPL to the S&P 500 in terms of total returns.

Final Answer: 
Insights and alerts about significant market opportunities or threats for AAPL:

After

Final Answer: 
Potential Trading Strategies for AAPL:
1. **Buy and Hold Strategy**: Investors with a long-term perspective and high risk tolerance may consider buying AAPL stock before the earnings date and holding it for an extended period to capitalize on potential future growth.
2. **Swing Trading Strategy**: Traders with a medium risk tolerance and short to medium-term trading preferences could consider entering a position in AAPL before the earnings date and aiming to capture short-term price movements within the identified uptrend.
3. **Options Trading Strategy**: For investors seeking to leverage the uptrend while managing risk, options strategies such as covered calls or protective puts could be employed around the earnings date to potentially enhance returns or protect against downside risk.
4. **Diversification**: It is crucial to diversify the trading strategies and not put all investments in a single stock. Considering overall market performance and diversifying the portfol

Final Answer: Page Not Found, unable to access the approved trading strategies for AAPL on Investopedia.

> Finished chain.
 

Page Not Found, unable to access the approved trading strategies for AAPL on Investopedia.

Final Answer: I apologize for the inconvenience, but I was unable to delegate the task to either the Data Analyst or the Trading Strategy Developer due to technical difficulties accessing the necessary information. As the Crew Manager, I will personally take on the task of analyzing the approved trading strategies for AAPL to determine the best execution methods considering current market conditions and optimal pricing. 

---

Detailed execution plans suggesting how and when to execute trades for AAPL:
1. **Buy and Hold Strategy**: 
   - **Execution Plan**: Invest in AAPL stock before the earnings date and hold it for an extended period.
   - **Timing**: Enter the position before the earnings date to capitalize on potential future growth.
   - **Risk Assessment**: High r

Final Answer: 
I was unable to access the specific content related to trading strategies for AAPL on the provided website. I recommend looking for alternative sources or seeking advice from industry experts to gather information on potential risks and mitigation strategies for the proposed trading strategies for AAPL.

> Finished chain.
 

I was unable to access the specific content related to trading strategies for AAPL on the provided website. I recommend looking for alternative sources or seeking advice from industry experts to gather information on potential risks and mitigation strategies for the proposed trading strategies for AAPL.


Thought: I need to gather more insights into the potential risks and mitigation strategies for the proposed trading strategies for AAPL to ensure a well-rounded analysis.

Action: Ask question to co-worker
Action Input: 
{
  "coworker": "Risk Advisor",
  "question": "What are the key risks associated with each of the proposed trading strategies for 

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

I apologize for not being able to provide a comprehensive risk analysis report detailing potential risks and mitigation recommendations for the proposed trading strategies for AAPL. Due to technical difficulties accessing the necessary information, I was unable to delegate the task to the Trade Advisor or the Risk Advisor. I recommend conducting further research on reputable sources or seeking advice from industry experts to gather insights into the potential risks associated with each trading strategy and suggest effective mitigation strategies. This will ensure a well-rounded analysis and enhance the team's success in executing the plans.